# Introduction

## Goal.
The goal of this lab is to study the application of quality-diversity algorithms, in particular the Multi-dimensional Archive of Phenotypic Elites (MAP-Elites) to various kinds of problems. We will also investigate the parametrization of the algorithm and its effect on the algorithmic performance.

## Getting started. 
In this lab we will use the `qdpy`$^{[1]}$ library, as well as the `deap` library seen in the previous lab. All the exercises are based on the examples available in the library $^{[2]}$, with some modifications. Please note that the `qdpy` library contains many other quality-diversity algorithms, such as Novelty Search, and advanced variants of MAP-Elites. However, in this lab we will use for simplicity only the vanilla version of MAP-Elites.

The basic version of MAP-Elites is shown in Algorithm 1. In the pseudo-code, $\textbf{x}$ and $\textbf{x}'$ are candidate solutions (i.e., $n$-dimensional vectors defined in the search space $\textbf{D}$); $\textbf{b}$ is a *feature descriptor*, that is a location in a user-defined *discretized* feature space (which can be seen as a *grid* made of *bins*), corresponding to the candidate solution $\textbf{x}'$, (i.e., an $N$-dimensional vector of user-defined features that characterize $\textbf{x}'$, typically with $N<n$); $p'$ is the performance of the candidate solution $\textbf{x}'$ (i.e., the scalar value returned by the objective function $f(\textbf{x}')$); $\mathcal{P}$ is a $<$ feature descriptor, performance $>$ map (i.e., an associative table that stores the best performance associated to each feature descriptor encountered by the algorithm); $\mathcal{X}$ is a $<$ feature descriptor, solution $>$ map (i.e., an associative table that stores the best solution associated to each feature descriptor encountered by the algorithm); $\mathcal{P}(\textbf{b}')$ is the best performance associated to the feature descriptor $\textbf{b}'$ (it can be empty); $\mathcal{X}(\textbf{b}')$ is the best solution associated to the feature descriptor $\textbf{b}'$ (it can be empty).

![alg1.png](img/img_12/alg1.png)

Following the pseudo-code shown above, the algorithm first creates the two maps $\mathcal{P}$ and $\mathcal{X}$, which are initially empty. Then, a while loop is executed until a given stop criterion is not met (usually, on the maximum number of function evaluations). Each iteration of the loop evaluates a *batch* of solutions. In the first batch, a given number of solutions are randomly sampled, see the `randomSolution()` function, in the search space $\textbf{D}$, which are used for initializing the two maps $\mathcal{P}$ and $\mathcal{X}$. Then, starting from the next iteration, solutions are first randomly selected from the current map $\mathcal{X}$, through the randomSelection() operator, and then perturbed according to the `randomVariation()` operator. For each new solution $\textbf{x}'$, the corresponding feature descriptor $\textbf{b}'$ and performance $p'$ are then evaluated. At this point, the two maps $\mathcal{P}$ and $\mathcal{X}$ are updated: if the performance associated to $\textbf{b}'$, $\mathcal{P}(\textbf{b}')$, is empty (which can happen if this is the first time that the algorithm generates a solution with that feature descriptor), or if it contains a value that is worse than the performance $p'$ of the newly generated solution (in the pseudo-code, we assume a minimization problem, therefore we check the condition $\mathcal{P}(\textbf{b}') > p'$), the new solution $\textbf{x}'$ and its performance $p'$ are assigned to the elements of the maps corresponding to its feature descriptor $\textbf{b}'$, namely $\mathcal{P}(\textbf{b}')$ and $\mathcal{X}(\textbf{b}')$. Once the loop terminates, the algorithm returns the two maps $\mathcal{P}$ and $\mathcal{X}$, which can be later analyzed for further inspection and post-processing.

It can be immediately noted how simple the algorithm is. With reference to the pseudo-code, in order to apply MAP-Elites to a specific problem the following methods must be defined:

 - $\textrm{randomSolution()}$: returns a randomly generated solution;
 - $\textrm{randomSelection}(\mathcal{X})$: randomly selects a solution from $\mathcal{X}$;
 - $\textrm{randomVariation}(\textbf{x})$: returns a modified copy of $\textbf{x}$;
 - $\textrm{featureDescriptor}(\textbf{x})$: maps a candidate solution $\textbf{x}$ to its feature descriptor, $\textbf{b}$;
 - $\textrm{performance}(\textbf{x})$: evaluates the objective function of the candidate solution $\textbf{x}$.

The first three methods are rather standard, i.e., they can be based on general-purpose operators typically used in EAs. However, it is possible to customize them according to the specific need. For instance, in the first two exercises of this lab we will use uniform random sampling and uniform random selection for the first two operators. For the variation operator, we will use the `RandomSearchMutPolyBounded` operator provided by `qdpy`, which essentially performs uniform random mutations with a saturation on the bounds of the search space. In the third exercise, we will use instead the typical operators of Genetic Programming.

As for what concerns $\textrm{featureDescriptor}(\textbf{x})$ and $\textrm{performance}(\textbf{x})$, these are obviously problem-dependent: the first one, being dependent on how the user defines the features of interest and the corresponding feature space; the latter, being dependent on the specific objective function at hand. In the exercises, we will see different definitions of performances and descriptors.

---
[1]: A Quality-Diversity framework for Python 3.6+: https://gitlab.com/leo.cazenille/qdpy

[2]: Examples avaiable at https://gitlab.com/leo.cazenille/qdpy/-/tree/master/examples

# Exercise 1
In this exercise, we will use MAP-Elites to _illuminate_ the feature space of a benchmark function that we have already used in some of the first labs, namely the Rastrigin function $^{[1]}$, which as you may remember is a highly multimodal problem.

For simplicity, we will use as feature descriptor for MAP-Elites the first two variables of the problem. Note however that, in general, the features used in MAP-Elites can be any property (different from the fitness function) of the solutions to the problem at hand.

To start the experiments, run the next cell. This will allow you to reproduce your results. At the end of the run, the script will generate a series of plots (see the figure below) in the `results/ex1/seed` directory, namely: 

 - `activityGrid.pdf`: this map indicates, for each bin, how many times that bin has been updated (i.e., its elite has been replaced) during the evolutionary process;
 - `evals_contsize.pdf`: this trend indicates the cumulative number of bins filled during the evolutionary process; 
 - `evals_fitnessmax0.pdf`: this trend is the usual fitness trend that we have seen in the previous labs (note: in this case the fitness has to be minimized);
 - `iterations_nbupdated.pdf`: this trend indicates how many bins are updated at each iteration of MAP-Elites;
 - `performancesGrid.pdf`: this is the final _illumination_ map that shows how the performance of the elites changes depending on the features at hand (brighter color indicates better results-note that the fitness is normalized in [0,1]).

The main outputs of the experiments of the first exercise are an *activity grid* (see the figure below, left), and a *performance grid* (see the figure below, right). More plots are available in the exercise folder after the execution of the experiments.

![ex1.png](img/img_12/ex1.png)

Furthermore, the script will serialize the final version of the map handled by MAP-Elites in a pickle file named `final.p`, that can be deserialized and manipulated for further analysis.

 - What kind of considerations can you make regarding the fitness trend (Is the algorithm able to converge to a reasonably low fitness function? How quick is the convergence?), and the activity grid (For instance, are there regions of the feature space that are visited/updated more frequently than others?). What kind of illumination pattern do you observe? Do you see any trend/correlation between performance and features of the map?
    
 - Try to change the parameters of the MAP-Elites algorithm, i.e.,: `NO_BINS`, `MAX_ITEMS_BIN`, `BUDGET`, `BATCH_SIZE`, which indicate, respectively, the number of bins (that is the same for both features), the maximum number of items stored in each bin of the grid, the total budget of the evolutionary process (number of function evaluations), and the batch size, i.e., how many solutions are evaluated at each iteration of MAP-Elites. Focus in particular on `NO_BINS`. What is the effect on the fitness trend and the performance map when you increase or decrease the number of bins?
    
 - Try to change the problem dimension (`PROBLEM_DIM`) to a much larger value, for instance 10 (remember that Rastrigin is a scalable benchmark problem, meaning that it can be defined for any number of variables). Note that in any case the first two variables are taken as features for MAP-Elites. What kind of considerations can you make in this case regarding the illumination pattern and the other aspects (i.e., the fitness trend and the activity grid) of the results? Does illumination become more difficult (i.e., less bins are visited, with poorer performance)? Why?

---

[1]: Rastrigin function https://pythonhosted.org/inspyred/reference.html\#inspyred.benchmarks.Rastrigin

In [ ]:
from utils.utils_12.exercise_rastrigin import main

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""
# More bins --> less activation for bin --> less competition --> preserve solutions potentially good?
# MOre dimensions -> first two dimensions as features capture less information --> worse performance

# TODO: change these parameters
NO_BINS = 32  # default 32
MAX_ITEMS_BIN = 1  # default 1
BUDGET = 10000  # default 10000
BATCH_SIZE = 500  # default 500
PROBLEM_DIM = 3  # default 3

"""
-------------------------------------------------------------------------
"""

main(NO_BINS, MAX_ITEMS_BIN, BUDGET, BATCH_SIZE, PROBLEM_DIM)

# Exercise 2
This exercise is similar to the previous one. The main difference is that in this case the objective function and feature descriptor are defined by a custom function, see `eval_fn`, that returns for each individual its fitness (`score`) and two features (`fit0` and `fit1`). Note that the fitness and features are based on trigonometric functions and are defined as scalable, i.e., they can be evaluated for any number of variables. 

To start the experiments, run the next cell. At the end of the run, the script will generate the same plots discussed in the previous exercise, as well as the pickle file containing the raw results, but it will save the results in the `results/ex2/seed` folder.


 - What kind of considerations can you make in this case regarding the fitness trend and illumination pattern?
 
 -  Also in this case, try to change `NO_BINS` and `PROBLEM_DIM`, and see if you can confirm the observations made in the previous experiment.
 
 - If you want, you could try to change the custom function definition in `eval_fn` and replicate the experiment with a different setting. What kind of results do you obtain?

In [ ]:
from utils.utils_12.exercise_custom_eval_fn import main
import math

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

# TODO: change these parameters
NO_BINS = 32  # default 32
MAX_ITEMS_BIN = 1  # default 1
BUDGET = 10000  # default 10000
BATCH_SIZE = 500  # default 500
PROBLEM_DIM = 3  # default 3


# TODO: try to define a different objective function (score) and/or features (fit0, fit1)
def eval_fn(ind):
    """An example evaluation function. It takes an individual as input, and returns the pair ``(fitness, features)``, where ``fitness`` and ``features`` are sequences of scores."""
    normalization = sum((x for x in ind))
    k = 10.0
    # score = k * len(ind) + sum((ind[i]**2 - k*math.cos(2*math.pi*ind[i] ) for i in range(len(ind)))) / float(len(ind))
    score = 1.0 - sum(
        (
            math.cos(k * ind[i]) * math.exp(-(ind[i] * ind[i]) / 2.0)
            for i in range(len(ind))
        )
    ) / float(len(ind))
    fit0 = sum((abs(x) * math.sin(abs(x) * 2.0 * math.pi) for x in ind)) / normalization
    # fit1 = sum((abs(x) * math.cos(abs(x) * 2. * math.pi) for x in ind)) / normalization
    fit1 = sum(math.exp(-(x * x) / 2.0) for x in ind) / normalization
    features = (fit0, fit1)
    return (score,), features


"""
-------------------------------------------------------------------------
"""
main(eval_fn, NO_BINS, MAX_ITEMS_BIN, BUDGET, BATCH_SIZE, PROBLEM_DIM)

# Exercise 3
In this exercise we will use MAP-Elites in combination with Genetic Programming (as implemented in `deap`, see the exercises from the previous lab) to solve a symbolic regression problem. The problem is exactly the same as the one seen in the first exercise of the previous lab on Genetic Programming (in which the goal was to fit a polynomial function), apart from the fact that in this case we will investigate how MAP-Elites illuminates a feature spaces characterized by the size of the tree (the number of nodes) and its depth. Similarly to the exercise seen in the lab on GP, the fitness in this case is the mean square error calculated on the training points (to be minimized). As both features can be seen as proxy for the complexity of the evolved trees, this experiments can give insights on how the performance of the trees depends on their complexity.

To start the experiments, run the next cell. Similarly to the previous exercises, at the end of the run the script will generate some plots (in this case only `activityGrid.pdf` and `performancesGrid.pdf`) as well as the pickle file containing the raw results. Also, note that the structure of the best tree is displayed on the terminal (in Reverse Polish notation).
Note, in this case, the file will be saved in the `results/ex3/seed` folder.

 - Is the algorithm able to approximate the given polynomial? If not, try to change some parameters of the algorithm and see if you can improve the results. Note that in this case there is an additional parameter (`INIT_BATCH_SIZE`), that is the size of the first batch (used to initialize the map). This is usually set to be bigger than the batch size at the subsequent iterations of the algorithm.
 - Try to change the generator function (e.g., to include trigonometric functions) defined in the method `generatorFunction`. Is the algorithm able to approximate more complicated generator functions? Which parameters can you change to improve the results?
 - What kind of illumination pattern do you observe in the various trials? Do you see any trend/correlation between performance and features (i.e., the size and depth of the tree)?

In [ ]:
#!/usr/bin/env python3
#    This file is part of qdpy.
#
#    qdpy is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    qdpy is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with qdpy. If not, see <http://www.gnu.org/licenses/>.


"""A simple example of MAP-elites to illuminate a fitness function
based on a symbolic regression problem. The illumination process
is ran with 2 features, i.e., the length of the tree (no. of nodes)
and its height (depth)."""

import matplotlib as mpl

mpl.use("Agg")
import matplotlib.pyplot as plt

from qdpy.algorithms.deap import *
from qdpy.containers import *
from qdpy.benchmarks import *
from qdpy.plots import *
from qdpy.base import *

from deap import base
from deap import creator
from deap import tools
from deap import gp
import operator

import os
import numpy as np
import random
import warnings

warnings.filterwarnings("ignore")

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

MAX_TREE_SIZE = 100  # default 20
MAX_ITEMS_BIN = 1  # default 1
INIT_BATCH_SIZE = 3000  # default 3000
BATCH_SIZE = 500  # default 500

GP_NGEN = 50  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.5, 1.0  # crossover and mutation probability for GP
seed = 21
parallelismType = "multithreading"  # type of parellism select one of this multiprocessing, concurrent, multithreading, scoop


# TODO: try to change the expression e.g. to include trigonometric functions
def generatorFunction(x):
    return math.sin(x) + math.cos(x)
    # return math.sin(x)*x**2
    # return math.sin(x)+5*x**2
    # return x**4 + x**3 + x**2 + x


"""
-------------------------------------------------------------------------
"""

# Create fitness classes (must NOT be initialised in __main__ if you want to use scoop)
fitness_weight = -1.0
creator.create("FitnessMin", base.Fitness, weights=(fitness_weight,))
creator.create(
    "Individual", gp.PrimitiveTree, fitness=creator.FitnessMin, features=list
)


def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1


def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # print(individual)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            # Compute tested function
            func_vals = np.array([func(x) for x in points])
            # Evaluate the mean squared error between the expression
            # and the target function
            sqerrors = (func_vals - ref_vals) ** 2.0
            fitness = [np.real(np.mean(sqerrors))]

        except Exception:
            fitness = [100.0]

    length = len(individual)
    height = individual.height
    features = [length, height]
    return [fitness, features]


# Compute reference function and stats
points = np.array(np.linspace(-1.0, 1.0, 1000), dtype=float)
dpoints = np.diff(points)
ref_vals = np.array([generatorFunction(x) for x in points])

# Create primitives
pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(operator.pow, 2)
# pset.addPrimitive(math.cos, 1) # TODO: uncomment this primitive if needed
# pset.addPrimitive(math.sin, 1) # TODO: uncomment this primitive if needed
try:
    pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
except:
    print("EphemeralConstant is already defined, if you changed it restart the kernel")
# pset.addEphemeralConstant("rand101", lambda: random.randint(-4.,4.))
pset.renameArguments(ARG0="x")

# Create Toolbox
max_size = MAX_TREE_SIZE
toolbox = base.Toolbox()
toolbox.register(
    "expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2
)  # NOTE: gen half/half initialization
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, points=points)
toolbox.register(
    "select", tools.selRandom
)  # NOTE: in MAP-Elites, random selection on a grid container
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate(
    "mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=max_size)
)
toolbox.decorate(
    "mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=max_size)
)

# Algorithm parameters
nb_bins = [max_size // 2, 10]  # The number of bins per feature
features_domain = [
    (1, max_size),
    (1, 10),
]  # The domain (min/max values) of the features
fitness_domain = [(0.0, np.inf)]  # The domain (min/max values) of the fitness
init_batch_size = INIT_BATCH_SIZE  # The number of evaluations of the initial batch ('batch' = population)
batch_size = BATCH_SIZE  # The number of evaluations in each subsequent batch
nb_iterations = (
    GP_NGEN  # The number of iterations (i.e. times where a new batch is evaluated)
)
cxpb = GP_CXPB  # The probability of mutating each value of a genome
mutation_pb = GP_MUTPB  # The probability of mutating each value of a genome
max_items_per_bin = MAX_ITEMS_BIN  # The number of items in each bin of the grid
verbose = True
show_warnings = True  # Display warning and error messages. Set to True if you want to check if some individuals were out-of-bounds
log_base_path = "./results/ex3/" + str(seed) + "/"
os.makedirs(log_base_path, exist_ok=True)
# Update and print seed
np.random.seed(seed)
random.seed(seed)
print("Seed: %i" % seed)
# Create a dict storing all relevant infos
results_infos = {}
results_infos["features_domain"] = features_domain
results_infos["fitness_domain"] = fitness_domain
results_infos["nb_bins"] = nb_bins
results_infos["init_batch_size"] = init_batch_size
results_infos["nb_iterations"] = nb_iterations
results_infos["batch_size"] = batch_size

# Create container
grid = Grid(
    shape=nb_bins,
    max_items_per_bin=max_items_per_bin,
    fitness_domain=fitness_domain,
    features_domain=features_domain,
    storage_type=list,
)

with ParallelismManager(parallelismType, toolbox=toolbox) as pMgr:
    # Create a QD algorithm
    algo = DEAPQDAlgorithm(
        pMgr.toolbox,
        grid,
        init_batch_size=init_batch_size,
        batch_size=batch_size,
        niter=nb_iterations,
        cxpb=cxpb,
        mutpb=mutation_pb,
        verbose=verbose,
        show_warnings=show_warnings,
        results_infos=results_infos,
        log_base_path=log_base_path,
    )
    # Run the illumination process !
    algo.run()

# Print results info
print(f"Total elapsed: {algo.total_elapsed}\n")
print(grid.summary())
# print("Best ever fitness: ", container.best_fitness)
# print("Best ever ind: ", container.best)
# print("%s filled bins in the grid" % (grid.size_str()))
# print("Solutions found for bins: ", grid.solutions)
# print("Performances grid: ", grid.fitness)
# print("Features grid: ", grid.features)

# Search for the smallest best in the grid:
smallest_best = grid.best
smallest_best_fitness = grid.best_fitness
smallest_best_length = grid.best_features[0]
interval_match = 1e-10
for ind in grid:
    if abs(ind.fitness.values[0] - smallest_best_fitness.values[0]) < interval_match:
        if ind.features[0] < smallest_best_length:
            smallest_best_length = ind.features[0]
            smallest_best = ind
print("Smallest best:", smallest_best)
print("Smallest best fitness:", smallest_best.fitness)
print("Smallest best features:", smallest_best.features)

# It is possible to access the results (including the genomes of the solutions, their performance, etc)
# stored in the pickle file by using the following code:
# ----8<----8<----8<----8<----8<----8<
# from deap import base, creator, gp
# import pickle
# fitness_weight = -1.0
# creator.create("FitnessMin", base.Fitness, weights=(fitness_weight,))
# creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin, features=list)
# pset = gp.PrimitiveSet("MAIN", 1)
# pset.addEphemeralConstant("rand101", lambda: random.randint(-4.,4.))
# with open("final.p", "rb") as f:
#    data = pickle.load(f)
# print(data)
# ----8<----8<----8<----8<----8<----8<
# --> data is a dictionary containing the results.

# Create plots
plot_path = os.path.join(log_base_path, "performancesGrid.pdf")
plotGridSubplots(
    grid.quality_array[..., 0],
    plot_path,
    plt.get_cmap("nipy_spectral"),
    grid.features_domain,
    grid.fitness_extrema[0],
    nbTicks=None,
)
print(
    "\nA plot of the performance grid was saved in '%s'." % os.path.abspath(plot_path)
)

plot_path = os.path.join(log_base_path, "activityGrid.pdf")
plotGridSubplots(
    grid.activity_per_bin,
    plot_path,
    plt.get_cmap("nipy_spectral"),
    grid.features_domain,
    [0, np.max(grid.activity_per_bin)],
    nbTicks=None,
)
print("\nA plot of the activity grid was saved in '%s'." % os.path.abspath(plot_path))

print("All results are available in the '%s' pickle file." % algo.final_filename)

# Instructions and questions

Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 

 - Do you think there is a trade-off between quality and diversity, or one aspect is more important than the other? If so, which one, and in which circumstances?
 
 - In which kind of applications do you think that MAP-Elites (and quality-diversity algorithms in general) could be useful? Why?